# Cargo librerías



In [23]:
import numpy as np #manejo de arreglos

import matplotlib.pyplot as plt #gráficos

from sklearn.decomposition import PCA #Componentes principales
from sklearn.preprocessing import StandardScaler #Escalado de datos

from sklearn.naive_bayes import GaussianNB #clasificador bayesiano ingenuo
# se pueden probar otros, por ej. MultinomialNB.
# La lista completa está acá: https://scikit-learn.org/stable/modules/naive_bayes.html

# matriz de confusión: https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Tamaño de gráficos
plt.rcParams["figure.figsize"] = (8,8)

# Datos y exploración

In [24]:
#https://www.kaggle.com/ronitf/heart-disease-uci
datos = np.loadtxt('./heart.csv',delimiter=',', skiprows=1)#, usecols = (0,2,3,4,5,6,7,8) )
nombres = ['age','sex','cp','rbs','sc','fbs120','recr','mhr','eia','op','slope','ca','thal']

# Hay veces conviene así, si los datos tiene faltantes
# https://www.kaggle.com/adityakadiwal/water-potability
# datos = np.genfromtxt('./water_potability.csv',delimiter=',', skip_header=1)#, usecols = (1,2,3,4) )
#elimina todas las filas con "nan"
# datos = datos[~np.isnan(datos).any(axis=1)]


# extraigo variables (predictoras, features, características, son todos nombres que se usan)
X = datos[:,0:-1]
# print(X.shape)
# print(X[0:5,:])

# extraigo clasificación (target, labels, etiquetas, son todos nombres que se usan)
y = datos[:,-1]
# print(y.shape)
# print(y[0:5])


# Escalo los datos: en general es buena idea para que el algoritmo funcione mejor
scaler = StandardScaler()
scaler.fit(X) #calcula promedio y desvío
X = scaler.transform(X) #estandariza con promedio y desvío antes calculado

# Clasificador bayesiano ingenuo

In [ ]:
gnb = GaussianNB() #instancio la clase
modelo_gnb = gnb.fit(X, y) #entreno modelo predictivo a partir de los datos, es decir construyo en borde de decisión
y_pred = modelo_gnb.predict(X) #clasifico según modelo. Por defecto clasifica según la clase con probablidad más alta

# Es una mala idea predecir con los mismos datos de entrenamiento: recordar que el objetivo es
# predecir la clasif. de datos NUEVOS no utilizados para entrenar el modelo.
# Por ahora lo dejamos así, pero pueden investigar si quieren y pueden.

# matriz de confusión
conf = confusion_matrix(y,y_pred)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels=gnb.classes_)
disp.plot(values_format='d') 

# Clasificación por punto de corte arbitrario

In [ ]:
probas = gnb.predict_proba(X) #probabilidades de cada clase según modelo predictivo
# inspecciono esas probabilidades
with np.printoptions(precision=3, suppress=True):
    print(probas[0:20])

# por ejemplo, por defecto asumo que SÍ tiene riesgo cardíco (y=1)
y_pred_07 = np.ones(y_pred.shape)

#y si la probabilidad de no tener es mayor a 0.7, lo clasifico como sin riesgo (y=0)
for i in range(probas.shape[0]):
    if (probas[i,0]>0.7):
        y_pred_07[i]=0.
# lo anterior se puede hacer más copado con listas por comprensión
# https://realpython.com/list-comprehension-python/
# https://entrenamiento-python-basico.readthedocs.io/es/latest/leccion10/listas_comprension.html


conf_07 = confusion_matrix(y,y_pred_07)

disp_07 = ConfusionMatrixDisplay(confusion_matrix=conf_07, display_labels=gnb.classes_)
disp_07.plot(values_format='d') 

# Visualizar resultados por Componentes principales

In [ ]:
pca = PCA()
pca.fit(X)
x_new = pca.transform(X)


def myplot(score,coeff,clases,labels=None):
    plt.figure()
    
    x1 = score[:,0]
    x2 = score[:,1]
    n = coeff.shape[0]

    scatter = plt.scatter(x1, x2, c=clases, alpha=0.5)#, cmap=colours)
    
    # print(scatter.legend_elements())
    # muestra etiquetas de las clases de la clasificación
    plt.legend(handles=scatter.legend_elements()[0], labels=scatter.legend_elements()[1], fontsize=16)

    # flechas de las proyecciones de las variables originales
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], color = 'r', alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center', fontsize=14)
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center', fontsize=14)



#Grafico las primeras dos componentes principales para cada clasificación

#clasificación real
myplot(x_new[:,0:2], pca.components_, y, nombres)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.title("Clasificación real")
plt.grid()
plt.show()

#clasificación bayes ingenuo
myplot(x_new[:,0:2], pca.components_, y_pred, nombres)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.title("Clasificación predicción Bayes ingenuo")
plt.grid()
plt.show()

#clasif. bayes ingenuo con punto de corte 0.7 para y=0
myplot(x_new[:,0:2], pca.components_, y_pred_07, nombres)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.title("Clasificación predicción Bayes ingenuo, con punto de corte 0.7 para y=0")
plt.grid()
plt.show()
